# RuleChef Tutorial

RuleChef takes labeled examples and uses an LLM to write regex rules that do the same job. Once the rules are learned, they run locally — no LLM needed, no API costs, sub-millisecond per query.

**Why rules instead of calling an LLM every time?**
- **Free to run** — no API calls at inference time
- **Fast** — under 1ms per query
- **Predictable** — same input always gives the same output
- **Transparent** — you can read, edit, and debug every rule

In this tutorial, we'll classify banking customer queries ("what's the exchange rate?" → `exchange_rate`) using 5 intent categories. We'll start with just a few examples and explore every tool RuleChef offers, then scale up to a real dataset.

## Part 1: Getting Started

In [ ]:
# Install (run once)
# !pip install rulechef datasets

In [ ]:
import os
import tempfile
from openai import OpenAI
from rulechef import RuleChef, Task, TaskType, RuleFormat

# --- LLM client ---
# Using Groq (free tier). Any OpenAI-compatible API works.
# For OpenAI: just OpenAI() with OPENAI_API_KEY set.
client = OpenAI(
    api_key=os.environ["OPENAI_API_KEY"],
    base_url="https://api.groq.com/openai/v1",
)
MODEL = "llama-3.1-8b-instant"

# --- Task definition ---
CLASSES = [
    "exchange_rate",
    "card_arrival",
    "beneficiary_not_allowed",
    "disposable_card_limits",
    "pending_cash_withdrawal",
]

task = Task(
    name="Banking Intent Classification",
    description=(
        f"Classify banking customer queries into one of these intents: {', '.join(CLASSES)}"
    ),
    input_schema={"text": "str"},
    output_schema={"label": "str"},
    type=TaskType.CLASSIFICATION,
    text_field="text",
)

# --- Create RuleChef ---
storage = tempfile.mkdtemp(prefix="rulechef_tutorial_")
chef = RuleChef(
    task,
    client,
    dataset_name="banking_tutorial",
    storage_path=storage,
    model=MODEL,
    allowed_formats=[RuleFormat.REGEX],  # regex-only for transparency
)
print(f"Storage: {storage}")

### Add examples

We'll start with just 5 examples — one per class. Each example is an input (`{"text": "..."}`) and the expected output (`{"label": "..."}`).

In [ ]:
examples = [
    ("what is the exchange rate for USD to EUR?", "exchange_rate"),
    ("my new card still hasn't arrived", "card_arrival"),
    ("why can't I send money to this person?", "beneficiary_not_allowed"),
    ("what are the limits on my disposable card?", "disposable_card_limits"),
    ("I withdrew cash but it's still pending", "pending_cash_withdrawal"),
]

for text, label in examples:
    chef.add_example({"text": text}, {"label": label})

# Examples sit in a buffer until we call learn_rules()
print(chef.get_buffer_stats())

### Learn rules

This sends the examples to the LLM, which writes regex rules. Then it tests them against the examples and patches any mistakes.

In [ ]:
rules, eval_result = chef.learn_rules()

## Part 2: Inspect Everything

### Rule summary

A quick overview of what was learned:

In [ ]:
for i, s in enumerate(chef.get_rules_summary(), 1):
    print(
        f"{i:2d}. {s['name']:<40s} format={s['format']}  priority={s['priority']}  confidence={s['confidence']}"
    )

### The actual regex patterns

The summary hides the regex. Let's look at the real rules — what regex matches, and what label it produces:

In [ ]:
for rule in chef.dataset.rules:
    label = rule.output_template.get("label", "?") if rule.output_template else "?"
    print(f"[{rule.id}] {rule.name}")
    print(f"  Pattern:  {rule.content}")
    print(f"  Outputs:  label={label}")
    print(f"  Priority: {rule.priority}")
    print()

### Try them out

In [ ]:
test_queries = [
    "current exchange rate please",
    "where is my card?",
    "I can't add a beneficiary",
    "disposable card spending limit",
    "still waiting on my cash withdrawal",
    "hello how are you",  # out-of-scope — should get no match
]

for q in test_queries:
    result = chef.extract({"text": q})
    print(f"{q:50s} → {result.get('label', '(no match)')}")

In [ ]:
# How fast is it? No LLM call — just regex matching.
%timeit chef.extract({"text": "what is the exchange rate?"})

### Evaluate

How well do the rules do on the training data? `evaluate()` runs every rule against every example and computes precision, recall, and F1 per class.

In [ ]:
eval_result = chef.evaluate()

### Per-rule metrics

Which rules are pulling their weight? `get_rule_metrics()` evaluates each rule in isolation — useful for finding dead rules (0 matches) or harmful ones (low precision).

In [ ]:
rule_metrics = chef.get_rule_metrics()

### Delete a rule

If a rule is dead (0 matches) or harmful, you can remove it:

In [ ]:
# Find a dead rule (0 matches) if any
dead = [m for m in rule_metrics if m.matches == 0]
if dead:
    print(f"Deleting dead rule: {dead[0].rule_name} (id={dead[0].rule_id})")
    chef.delete_rule(dead[0].rule_id)
    print(f"Rules remaining: {len(chef.dataset.rules)}")
else:
    print("No dead rules found — all rules match something.")

## Part 3: Improve the Rules

### Corrections

When the rules get something wrong, you can correct them. Corrections are the highest-value training signal — they show exactly where rules fail.

In [ ]:
# Try something that might get misclassified
test_input = {"text": "when will my card get here?"}
wrong_output = chef.extract(test_input)
print(f"Got: {wrong_output}")
print("Expected: card_arrival")

# Add a correction — tell RuleChef what the right answer is
chef.add_correction(
    test_input,
    model_output=wrong_output,
    expected_output={"label": "card_arrival"},
    feedback="Questions about when a card will arrive should be card_arrival",
)

In [ ]:
# Incremental learn — patch existing rules instead of starting from scratch
chef.learn_rules(incremental_only=True)

In [ ]:
# Test again — did it fix it?
result = chef.extract({"text": "when will my card get here?"})
print(f"Now: {result}")

### Feedback

Feedback is general guidance that steers rule generation without being a labeled example. It's injected into the LLM prompt as "user guidance".

In [ ]:
chef.add_feedback(
    "Queries mentioning 'rate', 'convert', or 'currency' are usually exchange_rate. "
    "Queries mentioning 'waiting', 'arrive', 'deliver' are usually card_arrival."
)
print(f"Feedback count: {chef.get_stats()['feedback']}")

In [ ]:
# Re-learn — feedback gets injected into the synthesis prompt
chef.learn_rules()

### Sampling strategies

When you have lots of examples, RuleChef can't fit them all in the LLM prompt. It picks a subset using a **sampling strategy**:

| Strategy | What it does |
|----------|-------------|
| `balanced` | Takes examples in order (default) |
| `recent` | Newest examples first |
| `diversity` | Evenly spaces picks across the dataset |
| `uncertain` | Low-confidence examples first |
| `varied` | Mix of recent + diverse + uncertain |
| `corrections_first` | Corrections always go first (they already do by default, but this also sorts examples by recency) |

All strategies always include corrections first — they're the highest-value signal.

You can set a default strategy when creating RuleChef, or override per call:

In [ ]:
# Override sampling for a single learn call
chef.learn_rules(sampling_strategy="diversity")

## Part 4: Under the Hood

What actually goes to the LLM? Let's look at the prompts RuleChef builds.

### The synthesis prompt

This is the prompt that asks the LLM to generate rules from examples. It includes:
1. **Task description** — what we're trying to do
2. **Data evidence** — patterns found in the examples (including auto-generated regex hints from `grex`)
3. **Training examples** — the actual input/output pairs
4. **User feedback** — any guidance you've added
5. **Format instructions** — how to write regex rules, with technique guides
6. **Response schema** — the exact JSON format to return

In [ ]:
# Build the synthesis prompt (what the LLM sees)
prompt = chef.learner._build_synthesis_prompt(chef.dataset, max_rules=10)
print(f"Prompt length: {len(prompt)} chars")
print("=" * 80)
print(prompt[:3000])
print("\n... (truncated) ...")

### The patch prompt

When rules don't match all examples, RuleChef runs a refinement loop. The patch prompt shows the LLM the current rules, the failures, and asks it to fix them:

In [ ]:
# Build a patch prompt — shows current rules + failures
# First, get some failures to show
from rulechef.evaluation import evaluate_dataset

eval_result = evaluate_dataset(
    chef.dataset.rules, chef.dataset, chef.learner._apply_rules
)

if eval_result.failures:
    patch_prompt = chef.learner._build_patch_prompt(
        chef.dataset.rules, eval_result.failures, max_rules=5, dataset=chef.dataset
    )
    print(f"Patch prompt length: {len(patch_prompt)} chars")
    print("=" * 80)
    print(patch_prompt[:3000])
    print("\n... (truncated) ...")
else:
    print("No failures — all examples matched! No patch prompt needed.")

### grex: regex hints from examples

[grex](https://github.com/pemistahl/grex) is a library that infers regex patterns from example strings. You give it strings, it gives you a regex that matches all of them. RuleChef uses this to give the LLM concrete pattern suggestions during rule synthesis.

Let's first see what grex does on its own, then see how RuleChef uses it in prompts.

In [ ]:
import re

try:
    from grex import RegExpBuilder

    # Example 1: Dates — grex finds the structural pattern
    dates = ["2024-01-15", "2024-02-28", "2023-12-01", "2025-06-30"]
    exact = RegExpBuilder.from_test_cases(dates).without_anchors().build()
    generalized = (
        RegExpBuilder.from_test_cases(dates)
        .without_anchors()
        .with_conversion_of_digits()
        .with_conversion_of_repetitions()
        .build()
    )
    print("=== Dates ===")
    print(f"  Input:      {dates}")
    print(f"  Exact:      {exact}")
    print(f"  Generalized: {generalized}")
    # Verify the generalized pattern matches new dates
    print(f"  Matches '2026-03-14'? {bool(re.search(generalized, '2026-03-14'))}")
    print()

    # Example 2: Dollar amounts
    amounts = ["$10.99", "$5.00", "$249.95", "$1.50"]
    exact = RegExpBuilder.from_test_cases(amounts).without_anchors().build()
    generalized = (
        RegExpBuilder.from_test_cases(amounts)
        .without_anchors()
        .with_conversion_of_digits()
        .with_conversion_of_repetitions()
        .build()
    )
    print("=== Dollar amounts ===")
    print(f"  Input:      {amounts}")
    print(f"  Exact:      {exact}")
    print(f"  Generalized: {generalized}")
    print()

    # Example 3: Natural language — exact pattern is just alternation
    phrases = ["exchange rate", "current rates", "conversion rate"]
    exact = RegExpBuilder.from_test_cases(phrases).without_anchors().build()
    print("=== Natural language phrases ===")
    print(f"  Input:      {phrases}")
    print(f"  Exact:      {exact}")
    print("  (No structural pattern — text is too diverse to generalize)")

except ImportError:
    print("grex not installed. Install with: pip install rulechef[grex]")

In [ ]:
# Extract just the data evidence section from the prompt
prompt_with_grex = chef.learner._build_synthesis_prompt(chef.dataset, max_rules=10)

# Find the data evidence section
start = prompt_with_grex.find("DATA EVIDENCE FROM TRAINING:")
end = prompt_with_grex.find("\n\n", start + 1) if start != -1 else -1
if start != -1:
    evidence_with = prompt_with_grex[start:end].strip()
    print("=== With grex (default) ===")
    print(evidence_with)
else:
    print("DATA EVIDENCE section not found — grex may not be installed.")
    print("Install with: pip install rulechef[grex]")

In [ ]:
# Now disable grex and compare
chef.learner.prompt_builder.use_grex = False
prompt_without_grex = chef.learner._build_synthesis_prompt(chef.dataset, max_rules=10)

start = prompt_without_grex.find("DATA EVIDENCE FROM TRAINING:")
end = prompt_without_grex.find("\n\n", start + 1) if start != -1 else -1
if start != -1:
    evidence_without = prompt_without_grex[start:end].strip()
    print("=== Without grex ===")
    print(evidence_without)
else:
    print("No DATA EVIDENCE section — this task type may not generate one.")

# Restore grex
chef.learner.prompt_builder.use_grex = True

# Show the difference
print("\n" + "=" * 60)
print(f"Prompt length WITH grex:    {len(prompt_with_grex):,} chars")
print(f"Prompt length WITHOUT grex: {len(prompt_without_grex):,} chars")
print(
    f"grex adds {len(prompt_with_grex) - len(prompt_without_grex):,} chars of regex hints"
)

## Part 5: Agentic Learning

The default coordinator uses simple heuristics. The **Agentic Coordinator** uses a second LLM call each iteration to analyze per-class metrics and tell the rule generator what to focus on. It also decides when to stop early.

In [ ]:
from rulechef.coordinator import AgenticCoordinator

# Add a few more examples so there's more to work with
more_examples = [
    ("how much is a dollar in euros?", "exchange_rate"),
    ("I want to know the current rates", "exchange_rate"),
    ("when will my new card be delivered?", "card_arrival"),
    ("my card hasn't come in the mail yet", "card_arrival"),
    ("it says beneficiary not allowed", "beneficiary_not_allowed"),
    ("I can't transfer money to my friend", "beneficiary_not_allowed"),
    ("what's the spending limit on virtual cards?", "disposable_card_limits"),
    ("how much can I spend on a disposable card?", "disposable_card_limits"),
    ("my ATM withdrawal is still processing", "pending_cash_withdrawal"),
    ("cash withdrawal hasn't gone through yet", "pending_cash_withdrawal"),
]

for text, label in more_examples:
    chef.add_example({"text": text}, {"label": label})

print(f"Buffer: {chef.get_buffer_stats()['new_examples']} new examples")

In [ ]:
# Switch to agentic coordinator
chef.coordinator = AgenticCoordinator(client, model=MODEL)

# Learn with more refinement iterations — watch the coordinator's guidance
rules, eval_result = chef.learn_rules(max_refinement_iterations=5)

In [ ]:
# Check the results
chef.evaluate()

### Rule pruning

After multiple refinement iterations, the ruleset can get bloated with redundant rules. Enable `prune_after_learn` and the coordinator will merge similar patterns and remove pure noise — with a safety net that reverts if quality drops.

In [ ]:
print(f"Rules before pruning: {len(chef.dataset.rules)}")

# Enable pruning
chef.coordinator = AgenticCoordinator(client, model=MODEL, prune_after_learn=True)

rules, eval_result = chef.learn_rules(max_refinement_iterations=3)

print(f"\nRules after pruning: {len(chef.dataset.rules)}")

## Part 6: LLM Observability

What if you already have an LLM classifying these queries in production? Instead of defining a task and manually labeling examples, you can **wrap your existing OpenAI client** and RuleChef will:

1. **Observe** every LLM call — capturing messages and responses
2. **Auto-discover** the task schema from the patterns it sees
3. **Map** observations to the discovered schema
4. **Learn rules** that replicate the LLM's behavior

```
Your code → wrapped_client.chat.completions.create(...) → LLM responds
                         ↓ (observed)
                   RuleChef captures raw interactions
                         ↓ (at learn time)
                   Discover task → Map to schema → Learn rules
```

**Zero-config**: no Task definition needed, no custom extractors. Just wrap and go.

In [ ]:
# Zero-config: no Task, no extractors — just observe
storage_obs = tempfile.mkdtemp(prefix="rulechef_observe_")
observer_chef = RuleChef(
    client=client,  # No task!
    dataset_name="banking_observed",
    storage_path=storage_obs,
    model=MODEL,
    allowed_formats=[RuleFormat.REGEX],
)

# Wrap the client — returns the same client object, now observed
wrapped_client = observer_chef.start_observing(client, auto_learn=False)
print("Observing... make some LLM calls.")

In [ ]:
# Simulate your existing LLM pipeline — classify some queries
# RuleChef silently captures every call (messages + response).
queries = [
    "what's the dollar to yen rate?",
    "I'm still waiting for my new card",
    "why is this beneficiary blocked?",
    "can I increase my disposable card limit?",
    "my cash withdrawal is stuck",
    "how do I convert euros to pounds?",
    "when will my replacement card arrive?",
    "what's the current USD exchange rate?",
]

system_prompt = (
    f"Classify this banking query into exactly one of these intents: {', '.join(CLASSES)}. "
    "Reply with just the intent label, nothing else."
)

for q in queries:
    response = wrapped_client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": q},
        ],
        temperature=0,
    )
    print(f"{q:50s} → {response.choices[0].message.content.strip()}")

# Check what was captured
print(f"\nObserver stats: {observer_chef.get_buffer_stats().get('observer', {})}")

In [ ]:
# learn_rules() does everything:
#   1. Discovers the task schema from raw observations (since no task was provided)
#   2. Maps each observation to the discovered schema
#   3. Commits mapped examples to buffer → dataset
#   4. Synthesizes + refines rules
rules, eval_result = observer_chef.learn_rules()

# Show what was discovered
print(f"\nDiscovered task: {observer_chef.task.name}")
print(f"  Type: {observer_chef.task.type.value}")
print(f"  Input schema: {observer_chef.task.input_schema}")
print(f"  Output schema: {observer_chef.task.output_schema}")

# Stop observing — restores the original client
observer_chef.stop_observing()

# The rules work without any LLM calls
for q in ["exchange rate for dollars", "where is my debit card?"]:
    result = observer_chef.extract({"text": q})
    print(f"  {q:40s} → {result}")

## Part 7: Scale Up

Now let's use a real dataset. [Banking77](https://huggingface.co/datasets/legacy-datasets/banking77) has 13,000 banking customer queries across 77 intent classes. We'll use the same 5 classes with 5 examples per class (25 total) and test on held-out data.

In [ ]:
import random
import uuid
from collections import defaultdict
from datasets import load_dataset
from rulechef.core import Dataset, Example

# Load Banking77
ds = load_dataset("legacy-datasets/banking77")
label_names = ds["train"].features["label"].names

train_records = [
    {"text": row["text"], "label": label_names[row["label"]]} for row in ds["train"]
]
test_records = [
    {"text": row["text"], "label": label_names[row["label"]]} for row in ds["test"]
]

print(
    f"Banking77: {len(train_records)} train, {len(test_records)} test, {len(label_names)} classes"
)

In [ ]:
# Sample 5-shot per class for our 5 classes
rng = random.Random(42)
by_label = defaultdict(list)
for ex in train_records:
    if ex["label"] in CLASSES:
        by_label[ex["label"]].append(ex)

train_sample = []
train_remaining = []
for label in sorted(CLASSES):
    exs = by_label[label]
    rng.shuffle(exs)
    train_sample.extend(exs[:5])
    train_remaining.extend(exs[5:])

# Filter test set to our 5 classes
test_data = [ex for ex in test_records if ex["label"] in CLASSES]

print(f"Training: {len(train_sample)} examples (5-shot x {len(CLASSES)} classes)")
print(f"Held-out eval: {len(train_remaining)} examples")
print(f"Held-out test: {len(test_data)} examples")

# Show a few
for ex in train_sample[:3]:
    print(f"  {ex['label']:30s} | {ex['text']}")

In [ ]:
# Create a fresh chef for the scale-up
storage2 = tempfile.mkdtemp(prefix="rulechef_banking77_")
chef2 = RuleChef(
    task,
    client,
    dataset_name="banking77_5shot",
    storage_path=storage2,
    model=MODEL,
    allowed_formats=[RuleFormat.REGEX],
)

# Add all 25 training examples
for ex in train_sample:
    chef2.add_example({"text": ex["text"]}, {"label": ex["label"]})

# Learn with evaluation
rules, eval_result = chef2.learn_rules(max_refinement_iterations=3)

### Evaluate on held-out test data

The rules were learned from 25 examples. Let's see how they do on data they've never seen:

In [ ]:
import time

# Build a test Dataset
test_dataset = Dataset(name="banking77_test", task=task)
for ex in test_data:
    test_dataset.examples.append(
        Example(
            id=str(uuid.uuid4())[:8],
            input={"text": ex["text"]},
            expected_output={"label": ex["label"]},
            source="benchmark",
        )
    )

# Evaluate
t0 = time.time()
test_eval = evaluate_dataset(
    chef2.dataset.rules, test_dataset, chef2.learner._apply_rules
)
elapsed = time.time() - t0

print(f"\nHeld-out test results ({len(test_data)} examples):")
print(f"  Accuracy:    {test_eval.exact_match:.1%}")
print(f"  Precision:   {test_eval.micro_precision:.1%}")
print(f"  Recall:      {test_eval.micro_recall:.1%}")
print(f"  Micro F1:    {test_eval.micro_f1:.1%}")
print(f"  Macro F1:    {test_eval.macro_f1:.1%}")
print(f"  Per query:   {elapsed / len(test_data) * 1000:.2f}ms")
print(f"  Rules:       {len(chef2.dataset.rules)}")

print("\nPer class:")
for cm in sorted(test_eval.per_class, key=lambda c: c.f1, reverse=True):
    print(
        f"  {cm.label:35s} F1={cm.f1:.0%}  P={cm.precision:.0%}  R={cm.recall:.0%}  (TP={cm.tp} FP={cm.fp} FN={cm.fn})"
    )

### Persistence

Rules are automatically saved to disk as JSON. You can reload them by creating a new `RuleChef` with the same `dataset_name` and `storage_path`.

In [ ]:
import json
from pathlib import Path

# Show the saved file
saved_file = Path(storage2) / "banking77_5shot.json"
data = json.loads(saved_file.read_text())
print(f"Saved to: {saved_file}")
print(f"File size: {saved_file.stat().st_size / 1024:.1f} KB")
print(
    f"Contains: {len(data.get('rules', []))} rules, {len(data.get('examples', []))} examples"
)

In [ ]:
# Recreate chef from disk — rules load automatically
chef3 = RuleChef(
    task,
    client,
    dataset_name="banking77_5shot",
    storage_path=storage2,
    model=MODEL,
    allowed_formats=[RuleFormat.REGEX],
)
print(f"Loaded {len(chef3.dataset.rules)} rules from disk")

# Works immediately
result = chef3.extract({"text": "what's the exchange rate for dollars?"})
print(f"Extract: {result}")

# Measure throughput
t0 = time.time()
for ex in test_data[:100]:
    chef3.extract({"text": ex["text"]})
elapsed = time.time() - t0
print(
    f"\n100 queries in {elapsed * 1000:.1f}ms ({elapsed / 100 * 1000:.2f}ms per query)"
)

## Summary

In this tutorial you learned:

1. **Define a task** — describe what you want to classify, give it labels
2. **Add examples** — just text + expected label
3. **Learn rules** — the LLM writes regex rules from your examples
4. **Inspect rules** — see the actual patterns, priorities, and output templates
5. **Extract** — classify new text with sub-ms latency, no LLM needed
6. **Evaluate** — measure precision/recall/F1 per class and per rule
7. **Correct mistakes** — add corrections and re-learn incrementally
8. **Give feedback** — steer rule generation with natural language guidance
9. **Sampling strategies** — control which examples the LLM sees
10. **Under the hood** — inspect synthesis and patch prompts, see how grex adds regex hints
11. **Agentic learning** — LLM-guided refinement that focuses on weak classes
12. **Rule pruning** — merge redundant rules with a safety net
13. **LLM observability** — wrap your existing LLM client, auto-discover the task schema, and learn rules without any manual labeling
14. **Scale up** — from 5 examples to a real dataset
15. **Persistence** — rules save to disk and reload automatically

**What to do next:**
- Add more examples from production data
- Use corrections when users report mistakes
- Try more classes (Banking77 has 77!)
- Use `RuleFormat.CODE` for more complex logic
- Use the CLI for interactive exploration: `rulechef`